# Notebook to process IACMI stress-strain curve data

This is useful for generating stress-strain information on the MTS 370.10 and 370.02 in W230 using an extensometer.

In [28]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt 
from scipy import stats 
import shutil
from io import StringIO
import statsmodels.formula.api as smf

## User input

- force/strain filename
- sample cross sectional area

In [29]:
# Setting the directory
os.chdir('/Users/nicolethomas/Google Drive/Grad School/IACMI Project/Python')

# Set file name
filename = "tensile.csv"

# Set results output filename
Results = "tensile_results.csv"

# Input dimensions and calculate area
b=float(input('Input the width of the sample in millimeters:'))/1000
print (b)
h=float(input('Input the thickness of the sample in millimeters:'))/1000
Area = b*h

print (("Cross-sectional area= %f m^2") %Area)

#Aruvi's area code: Area = int(Dimensions["Width"]*Dimensions["Thickness"])

Input the width of the sample in millimeters:15
0.015
Input the thickness of the sample in millimeters:4
Cross-sectional area= 0.000060 m^2


## Read input file

The time, force, displacement and extensometer information is provided in a single file since the extensometer is connected to the MTS software. This is read into a pandas DataFrame.

In [30]:
def scrub_file(filename, separator=',', skip_header=0):
    # initialize variables
    line = ""
    contents = ""
    lineno = 0
    numfields = 0
    # what conversion are we performing? what does it return if the
    # conversion fails?
    def tofloat(x):
        try:
            return float(x)
        except ValueError:
            return np.nan
    # this is going to handle reading a line from the file, and automatically
    # do anything else we need, such as keep track of which line number
    # we're on.
    def readline():
        nonlocal line, fstream
        #global line
        line = fstream.readline()
        #lineno += 1
        return line
    # set the converter
    convert = tofloat
    # read the file
    with open(filename) as fstream:
        # pass header rows straight through
        for _ in range(skip_header):
            contents.append(readline(fstream))
        # check how many fields fail conversion
        while readline():
            # process the line into fields (comma-separated by default)
            fields = [convert(field) for field in line.split(separator)]
            # how many of these fields are not NaN when an attempt is made
            # to convert them to a float?
            notnan = np.sum(~np.isnan(fields))
            # **This is where all the work is done!** How do you know if a row is
            # bad? Is it bad if all fields fail the conversion? All-but-one? All-but-two?
            # You have to look at the source file and make this determination, then
            # change this inequality. Only rows that pass the test are kept.
            if notnan > 0:
                contents += line
            # update the numfields variable -- this tracks the row with the most
            # number of fields.
            numfields = max(notnan, numfields)
    # now return the good lines (a.k.a. records) as a StringIO object. (StringIO
    # allows you to read from a string object as if it were a file.)
    return StringIO(contents)

# now use the output from scrub_file in your earlier code, e.g.
tensile = pd.read_csv(scrub_file(filename),
    usecols=[2,4],
    names = ['Axial Extensometer', 'Axial Force']).dropna(axis = 0)

# check input file info
print('Input_size:')
print(tensile.shape)
print(tensile.head(5))

Input_size:
(1168, 2)
   Axial Extensometer  Axial Force
1           -0.000164    681.18506
2           -0.000153    680.99847
3           -0.000233    679.72717
4           -0.000193    679.57990
5           -0.000287    680.89185


## Process the input data

- calculate stress data from force and cross sectional area
- calculate strain data (no conversion, just extensometer input)

In [31]:
#Find the maximum force (in N) and calculste the max stress (in MPa)
Max_load=tensile['Axial Force'].values.max()
Stress = tensile["Axial Force"]/(1000000*Area)

#Read in strain data (already formatted in mm/mm)
Strain = tensile["Axial Extensometer"]

#print(tensile["Stress"].head(5))
print (('The max load on your sample is %f N') %Max_load)
print (('The tensile strength of your sample is %f MPa') %Stress.max()) 

print(Stress.head(5))
print(Strain.head(5))

The max load on your sample is 44460.102000 N
The tensile strength of your sample is 741.001700 MPa
1    11.353084
2    11.349975
3    11.328786
4    11.326332
5    11.348197
Name: Axial Force, dtype: float64
1   -0.000164
2   -0.000153
3   -0.000233
4   -0.000193
5   -0.000287
Name: Axial Extensometer, dtype: float64


# Plot stress vs. strain

Sanity check to ensure everything looks OK.

- Is there a compliance region?
- Do we need to eliminate any outliers (data collected after failure).

In [33]:
#Remove data after break
#for strains <0, delete entries
Strain_data = []
Stress_data = []

for i in range(0, len(Strain)):
    if Strain[i] > 0:
        Strain_data.append(Strain[i])
        Stress_data.append(Stress[i])

Data = pd.DataFrame({'Stress':Stress_data,'Strain':Strain_data},columns = ['Stress','Strain'])
print(Data.head(5))

KeyError: 0

## Calculate mechanical properties

- Young's modulus (according to ISO 527)
    - Fixed strain range, 0.005 to 0.025 mm/mm
    - Linear regression
        - Slope
        - $\sigma_{slope}$
        - Stress at failure
        - Strain at failure